In [ ]:
retail_graph_path = ""
business_graph_path = ""
nodes_path = ""

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

os.chdir(os.environ["EXPERIMENTS_HOMEDIR"])

%matplotlib inline

In [ ]:
business_g = nx.read_gpickle(business_graph_path)
retail_g = nx.read_gpickle(retail_graph_path)
nodes_df = pd.read_csv(nodes_path)

print("Business graph")
print(f"Num nodes: {len(business_g.nodes())}")
print(f"Num edges: {len(business_g.edges())}")
print("Retail graph")
print(f"Num nodes: {len(retail_g.nodes())}")
print(f"Num edges: {len(retail_g.edges())}")

In [ ]:
retail_edges_df = pd.DataFrame(
    [(e[0], e[1], e[2]["due_date"], e[2]["label"], e[2]["value"], e[2]["value_percentage"], e[2]["publish_date"]) for e in retail_g.edges(data=True)],
    columns=["from", "to", "due_date", "label", "value", "value_percentage", "publish_date"]
)
retail_edges_df["due_date"] = pd.to_datetime(retail_edges_df["due_date"])
retail_edges_df["due_date_year"] = retail_edges_df.due_date.apply(lambda dd: dd.year)

business_edges_df = pd.DataFrame(
    [(e[0], e[1], e[2]["due_date"], e[2]["label"], e[2]["value"], e[2]["value_percentage"], e[2]["publish_date"]) for e in business_g.edges(data=True)],
    columns=["from", "to", "due_date", "label", "value", "value_percentage", "publish_date"]
)
business_edges_df["due_date"] = pd.to_datetime(business_edges_df["due_date"])
business_edges_df["due_date_year"] = business_edges_df.due_date.apply(lambda dd: dd.year)

In [ ]:
print(round(business_edges_df.value.mean(), 2))
print(round(retail_edges_df.value.mean()), 2)

In [ ]:
retail_hist_df = retail_edges_df[
    ["due_date_year", "due_date"]
].groupby("due_date_year").count()
retail_hist_df = retail_hist_df / retail_hist_df.due_date.sum() * 100
retail_hist_df = retail_hist_df.rename(columns={"due_date": "Individuals"})

business_hist_df = business_edges_df[
    ["due_date_year", "due_date"]
].groupby("due_date_year").count()
business_hist_df = business_hist_df / business_hist_df.due_date.sum() * 100
business_hist_df = business_hist_df.rename(columns={"due_date": "Companies"})

In [ ]:
retail_hist_df.merge(
    business_hist_df, left_index=True, right_index=True, how="left"
).plot(kind="bar", figsize=(12,4), rot=45)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel("Maturity year", fontsize=18, labelpad=6)
plt.ylabel("% of receivables", fontsize=18, labelpad=5)
plt.legend(loc='upper right', fontsize=18, bbox_to_anchor=(0.3, 1.00));